<a href="https://colab.research.google.com/github/akshay659/Amazon-capstone-project/blob/main/Amazon_capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import gzip
import nltk
import re
nltk.download("stopwords")
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
df_beauty= getDF('/content/reviews_Beauty_5.json.gz')
df_grocery= getDF('/content/reviews_Grocery_and_Gourmet_Food_5.json.gz')
df_office= getDF('/content/reviews_Office_Products_5.json.gz')
df_pet=getDF('/content/reviews_Pet_Supplies_5.json.gz')
df_toys= getDF('/content/reviews_Toys_and_Games_5.json.gz')
df_home=getDF('/content/reviews_Home_and_Kitchen_5.json.gz')

In [ ]:
df_beauty['category']=0
df_grocery['category']=1
df_office['category']=2
df_pet['category']=3
df_toys['category']=4
df_home['category']=5

In [ ]:
df_beauty=df_beauty[['reviewText','summary','category']]
df_grocery=df_grocery[['reviewText','summary','category']]
df_office=df_office[['reviewText','summary','category']]
df_pet=df_pet[['reviewText','summary','category']]
df_toys=df_toys[['reviewText','summary','category']]
df_home=df_home[['reviewText','summary','category']]

In [ ]:
df_pet.head()

,reviewText,summary,category
0,I purchased the Trilogy with hoping my two cat...,Nice Distraction for my cats for about 15 minutes,3
1,There are usually one or more of my cats watch...,Entertaining for my cats,3
2,I bought the triliogy and have tested out all ...,Entertaining,3
3,My female kitty could care less about these vi...,Happy to have them,3
4,"If I had gotten just volume two, I would have ...",You really only need vol 2,3


In [ ]:
df_beauty.head()

,reviewText,summary,category
0,Very oily and creamy. Not at all what I expect...,Don't waste your money,0
1,This palette was a decent price and I was look...,OK Palette!,0
2,The texture of this concealer pallet is fantas...,great quality,0
3,I really can't tell what exactly this thing is...,Do not work on my face,0
4,"It was a little smaller than I expected, but t...",It's okay.,0


In [ ]:
df=pd.concat([df_beauty,df_grocery,df_office,df_pet,df_toys,df_home])

In [ ]:
df.head()

,reviewText,summary,category
0,Very oily and creamy. Not at all what I expect...,Don't waste your money,0
1,This palette was a decent price and I was look...,OK Palette!,0
2,The texture of this concealer pallet is fantas...,great quality,0
3,I really can't tell what exactly this thing is...,Do not work on my face,0
4,"It was a little smaller than I expected, but t...",It's okay.,0


In [ ]:
df.shape

(1280129, 3)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df['text']=df['reviewText']+df['summary']

In [ ]:
df.head()

,reviewText,summary,category,text
0,Very oily and creamy. Not at all what I expect...,Don't waste your money,0,Very oily and creamy. Not at all what I expect...
1,This palette was a decent price and I was look...,OK Palette!,0,This palette was a decent price and I was look...
2,The texture of this concealer pallet is fantas...,great quality,0,The texture of this concealer pallet is fantas...
3,I really can't tell what exactly this thing is...,Do not work on my face,0,I really can't tell what exactly this thing is...
4,"It was a little smaller than I expected, but t...",It's okay.,0,"It was a little smaller than I expected, but t..."


In [ ]:
df=df[['text','category']]

In [ ]:
df.head()

,text,category
0,Very oily and creamy. Not at all what I expect...,0
1,This palette was a decent price and I was look...,0
2,The texture of this concealer pallet is fantas...,0
3,I really can't tell what exactly this thing is...,0
4,"It was a little smaller than I expected, but t...",0


In [ ]:
# Text-Cleaning

stops=set(stopwords.words('english'))

def text_cleaning(sent):
  sent=sent.lower()
  sents=re.sub('[^a-z]',' ',sent)
  words=sents.split()
  imp_words=[w for w in words if w not in stops]
  return ' '.join(imp_words)

In [ ]:
df['clean_text']=df['text'].apply(lambda x: text_cleaning(x))


In [ ]:
df.head()


,text,category,clean_text
0,Very oily and creamy. Not at all what I expect...,0,oily creamy expected ordered try highlight con...
1,This palette was a decent price and I was look...,0,palette decent price looking different shades ...
2,The texture of this concealer pallet is fantas...,0,texture concealer pallet fantastic great cover...
3,I really can't tell what exactly this thing is...,0,really tell exactly thing powder kind oil ish ...
4,"It was a little smaller than I expected, but t...",0,little smaller expected okay lasted long time ...


In [ ]:
df = df[['clean_text','category']]

In [ ]:
df.dropna(axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
x = df['clean_text']
x.head()

0    oily creamy expected ordered try highlight con...
1    palette decent price looking different shades ...
2    texture concealer pallet fantastic great cover...
3    really tell exactly thing powder kind oil ish ...
4    little smaller expected okay lasted long time ...
Name: clean_text, dtype: object

In [ ]:
y = df['category']
y.head()

0    0
1    0
2    0
3    0
4    0
Name: category, dtype: int64

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=100,test_size=2)

In [ ]:
# Tokenization

tok=Tokenizer(oov_token='<unk>')

In [ ]:
tok.fit_on_texts(x_train)

In [ ]:
len(tok.word_index)

323722

In [ ]:
vocab_size=len(tok.word_index)+1

In [ ]:
np.mean(df['clean_text'].apply(lambda x : len(x.split())))

50.53376504972238

In [ ]:
x_train_sequences = tok.texts_to_sequences(x_train)
x_tets_sequences = tok.texts_to_sequences(x_test)

In [ ]:
max_length=100
embedding_dimension=300
padding_type='post'
truncate='post'

x_train_pad=pad_sequences(x_train_sequences,maxlen=max_length,padding=padding_type,truncating=truncate)
x_test_pad=pad_sequences(x_tets_sequences,maxlen=max_length,padding=padding_type,truncating=truncate)

In [ ]:
#model_Building


lstm_model=Sequential([tf.keras.layers.Embedding(vocab_size,embedding_dimension),
                       LSTM(128), #128 neurons in each network(usually selected in a power of 2)
                       Dense(50,activation='relu'),
                       Dense(6,activation='softmax')
])


In [ ]:
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 300)         97116900  
                                                                 
 lstm (LSTM)                 (None, 128)               219648    
                                                                 
 dense (Dense)               (None, 50)                6450      
                                                                 
 dense_1 (Dense)             (None, 6)                 306       
                                                                 
Total params: 97,343,304
Trainable params: 97,343,304
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm_model.compile(optimizer='adam',loss='SparseCategoricalCrossentropy',metrics=['accuracy'])

In [ ]:
history=lstm_model.fit(x_train_pad,y_train,validation_data=(x_test_pad,y_test),epochs=5,batch_size=1000)

Epoch 1/5
 584/1281 [============>.................] - ETA: 50:51 - loss: 1.3749 - accuracy: 0.4803

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Plot')
plt.legend(['Training Curve','Testing Curve']))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.legend(['Training Curve','Testing Curve'])